In [1]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [2]:
val_data = pd.read_csv('val.csv')
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_data.head()

,Phrase,Sentiment
0,"I mean, who needs sleep when you can have the ...",-100
1,"I'm not sure what's more disturbing, the fact ...",-100
2,a neon green jumpsuit with plaid pants. I mean...,-100
3,"It's just 'good luck, shut up",-100
4,"clearly, i'm a functioning adul",1


In [6]:
train_data['Phrase'] = train_data['Phrase'].str.lower()
train_data['Phrase'] = train_data['Phrase'].str.replace(r'[^\w\s]', '', regex=True)
train_data['Phrase'] = train_data['Phrase'].str.replace(r'http\S+', '', regex=True)
train_data = train_data.dropna()
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Define a function to safely lemmatize
def lemmatize_phrase(phrase):
    if isinstance(phrase, str):  # Check if phrase is a string
        return ' '.join([lemmatizer.lemmatize(word) for word in phrase.split()])
    return phrase  # Return the original value if it's not a string

train_data['Phrase'] = train_data['Phrase'].apply(lemmatize_phrase)

M = 5

vectorizer = CountVectorizer(binary=True, min_df=M)

X_train = vectorizer.fit_transform(train_phrases)

In [7]:
train_data.head()

,Phrase,Sentiment
0,i mean who need sleep when you can have the sa...,-100
1,im not sure whats more disturbing the fact tha...,-100
2,a neon green jumpsuit with plaid pant i mean w...,-100
3,it just good luck shut up,-100
4,clearly im a functioning adul,1


In [9]:
train_phrases = train_data['Phrase']

In [11]:
unlbl_data = train_data[train_data['Sentiment'] == -100]
lbl_data = train_data[train_data['Sentiment'] != -100]

In [13]:
X = lbl_data['Phrase']
y = lbl_data['Sentiment']  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline that combines TF-IDF and Logistic Regression with increased max_iter
model = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=200, solver='saga'))

model.fit(X_train, y_train)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=200, solver='saga'))])

In [22]:
y_pred = model.predict(X_val)

In [23]:
from sklearn.metrics import f1_score, accuracy_score

f1 = f1_score(y_val, y_pred, average='weighted')  # Use 'weighted' for multi-class classification
# Calculate Accuracy Score
accuracy = accuracy_score(y_val, y_pred)

# Print the scores
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy Score: {accuracy:.4f}')

F1 Score: 0.8308
Accuracy Score: 0.8307


In [24]:
model.fit(X, y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=200, solver='saga'))])

In [25]:
y_pred = model.predict(X_val)

In [26]:
f1 = f1_score(y_val, y_pred, average='weighted')  # Use 'weighted' for multi-class classification
# Calculate Accuracy Score
accuracy = accuracy_score(y_val, y_pred)

# Print the scores
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy Score: {accuracy:.4f}')

F1 Score: 0.8949
Accuracy Score: 0.8950


In [27]:
prediction = model.predict(unlbl_data['Phrase'])

In [28]:
prediction

array([3, 0, 3, ..., 4, 3, 4])

In [29]:
unlbl_data['Sentiment'] = prediction

/var/folders/g4/xw1m66k94_5glvd0271lzszr0000gn/T/ipykernel_21196/3850537874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlbl_data['Sentiment'] = prediction


In [30]:
unlbl_data.head()

,Phrase,Sentiment
0,i mean who need sleep when you can have the sa...,3
1,im not sure whats more disturbing the fact tha...,0
2,a neon green jumpsuit with plaid pant i mean w...,3
3,it just good luck shut up,3
5,yet in the memory we hold they remain a remind...,4


In [35]:
combined_data = pd.concat([lbl_data, unlbl_data], ignore_index=False)

In [36]:
combined_data.head()

,Phrase,Sentiment
4,clearly im a functioning adul,1
6,and it not what we thought the truth is out,0
7,guess ill just wing it and hope for the best,3
9,it not like ive been eating pizza and ice crea...,3
15,keep being extra yall,1


In [37]:
len(combined_data)

59701

In [38]:
X = combined_data['Phrase']
y = combined_data['Sentiment']

In [39]:
model.fit(X, y)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=200, solver='saga'))])

In [40]:
val_data['Phrase'] = val_data['Phrase'].str.lower()
val_data['Phrase'] = val_data['Phrase'].str.replace(r'[^\w\s]', '', regex=True)
val_data['Phrase'] = val_data['Phrase'].str.replace(r'http\S+', '', regex=True)
val_data = val_data.dropna()

lemmatizer = WordNetLemmatizer()

# Define a function to safely lemmatize
def lemmatize_phrase(phrase):
    if isinstance(phrase, str):  # Check if phrase is a string
        return ' '.join([lemmatizer.lemmatize(word) for word in phrase.split()])
    return phrase  # Return the original value if it's not a string

# Apply the function to the Phrase column
val_data['Phrase'] = val_data['Phrase'].apply(lemmatize_phrase)

# Parameters
M = 5

# Vectorization
vectorizer = CountVectorizer(binary=True, min_df=M)

In [41]:
X_val = val_data['Phrase']
y_val = val_data['Sentiment']

In [42]:
y_pred = model.predict(X_val)

In [43]:
f1 = f1_score(y_val, y_pred, average='weighted')  # Use 'weighted' for multi-class classification
accuracy = accuracy_score(y_val, y_pred)

print(f'F1 Score: {f1:.4f}')
print(f'Accuracy Score: {accuracy:.4f}')

F1 Score: 0.9121
Accuracy Score: 0.9122


In [44]:
test_data['Phrase'] = test_data['Phrase'].str.lower()
test_data['Phrase'] = test_data['Phrase'].str.replace(r'[^\w\s]', '', regex=True)
test_data['Phrase'] = test_data['Phrase'].str.replace(r'http\S+', '', regex=True)
test_data = test_data.dropna()

lemmatizer = WordNetLemmatizer()

# Define a function to safely lemmatize
def lemmatize_phrase(phrase):
    if isinstance(phrase, str):  # Check if phrase is a string
        return ' '.join([lemmatizer.lemmatize(word) for word in phrase.split()])
    return phrase  # Return the original value if it's not a string

# Apply the function to the Phrase column
test_data['Phrase'] = test_data['Phrase'].apply(lemmatize_phrase)

In [49]:
train_val = pd.concat([combined_data, val_data], ignore_index=False)

In [50]:
model.fit(train_val['Phrase'], train_val['Sentiment'])

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=200, solver='saga'))])

In [51]:
prediction = model.predict(test_data['Phrase'])

In [52]:
prediction

array([3, 2, 4, ..., 2, 3, 3])

In [54]:
pred = pd.DataFrame(prediction)

In [55]:
pred

,0
0,3
1,2
2,4
3,1
4,4
...,...
23246,2
23247,4
23248,2
23249,3


In [56]:
pred.value_counts()

2    4801
3    4771
4    4665
1    4551
0    4463
dtype: int64

In [57]:
test_data

,Phrase,PhraseID
0,i woke up got out of bed and managed to put on...,0
1,the human nose can detect over 1 trillion diff...,1
2,every pixel tell a story every brushstroke a m...,2
3,like im a busy person okay i have a life not a...,3
4,the smell of old book and coffee fill my lung ...,4
...,...,...
23252,learn about common allergen symptom and safety...,23252
23253,where will the horizon take me one day ill fin...,23253
23254,establish a bedtime routine create a sleepcond...,23254
23255,after careful consideration ive decided to wea...,23255


In [61]:
submission_df = pd.DataFrame({
    'PhraseID': test_data['PhraseID'],  # Assuming 'PhraseID' is the correct column name in test.csv
    'Sentiment': prediction
})

In [63]:
submission_df

,PhraseID,Sentiment
0,0,3
1,1,2
2,2,4
3,3,1
4,4,4
...,...,...
23252,23252,2
23253,23253,4
23254,23254,2
23255,23255,3


In [64]:
sample = pd.read_csv('sample_submission.csv')

In [66]:
sample

,PhraseID,Sentiment
0,0,2
1,1,2
2,2,2
3,3,2
4,4,2
...,...,...
23252,23252,2
23253,23253,2
23254,23254,2
23255,23255,2


In [67]:
test_data

,Phrase,PhraseID
0,i woke up got out of bed and managed to put on...,0
1,the human nose can detect over 1 trillion diff...,1
2,every pixel tell a story every brushstroke a m...,2
3,like im a busy person okay i have a life not a...,3
4,the smell of old book and coffee fill my lung ...,4
...,...,...
23252,learn about common allergen symptom and safety...,23252
23253,where will the horizon take me one day ill fin...,23253
23254,establish a bedtime routine create a sleepcond...,23254
23255,after careful consideration ive decided to wea...,23255


In [70]:
test = pd.read_csv('test.csv')

In [71]:
test

,Phrase,PhraseID
0,"I woke up, got out of bed, and managed to put ...",0
1,The human nose can detect over 1 trillion diff...,1
2,"Every pixel tells a story, every brushstroke a...",2
3,"Like, I'm a busy person, okay? I have a life, ...",3
4,The smell of old books and coffee fills my lun...,4
...,...,...
23252,"Learn about common allergens, symptoms, and sa...",23252
23253,"Where will the horizon take me? One day, I'll ...",23253
23254,"Establish a bedtime routine, create a sleep-co...",23254
23255,"After careful consideration, I've decided to w...",23255


In [72]:
# Step 1: Get the PhraseIDs from the submission DataFrame
submitted_ids = set(submission_df['PhraseID'])

# Step 2: Get the PhraseIDs from the original data
original_ids = set(test['PhraseID'])

# Step 3: Find missing PhraseIDs
missing_ids = original_ids - submitted_ids

print("Missing PhraseIDs:", missing_ids)

Missing PhraseIDs: {7498, 10762, 124, 16657, 13945, 11676}


In [74]:
missing_rows = test[test['PhraseID'].isin(missing_ids)]

missing_rows['Sentiment'] = 0  # or

/var/folders/g4/xw1m66k94_5glvd0271lzszr0000gn/T/ipykernel_21196/327352226.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_rows['Sentiment'] = 0  # or


In [75]:
submission_df = pd.concat([submission_df, missing_rows], ignore_index=True)

In [76]:

submission_df = submission_df.sort_values(by='PhraseID').reset_index(drop=True)

In [77]:
submission_df

,PhraseID,Sentiment,Phrase
0,0,3,NaN
1,1,2,NaN
2,2,4,NaN
3,3,1,NaN
4,4,4,NaN
...,...,...,...
23252,23252,2,NaN
23253,23253,4,NaN
23254,23254,2,NaN
23255,23255,3,NaN


In [80]:
submission_df[submission_df['PhraseID'] == 124]

,PhraseID,Sentiment,Phrase
124,124,0,NaN


In [81]:
submission_df = submission_df.drop(columns=['Phrase'])

In [82]:
submission_df

,PhraseID,Sentiment
0,0,3
1,1,2
2,2,4
3,3,1
4,4,4
...,...,...
23252,23252,2
23253,23253,4
23254,23254,2
23255,23255,3


In [84]:
sample

,PhraseID,Sentiment
0,0,2
1,1,2
2,2,2
3,3,2
4,4,2
...,...,...
23252,23252,2
23253,23253,2
23254,23254,2
23255,23255,2


In [85]:
# submission_df.to_csv('submissions.csv', index=False)